# import packges

this is continued from dataset/stJude.ipynb

In [6]:
library(DESeq2)
library(data.table)
library(dplyr)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(clusterProfiler)
library(AnnotationDbi)
library(org.Hs.eg.db)
library(GSVA)
library(ggplot2)
library(gridExtra)
library(RColorBrewer)
library(IRdisplay)
library(GGally)
library(ChIPpeakAnno)
library(universalmotif)
library(IRdisplay)
library(ggpubr)
library(DiffBind)
library(kableExtra)
library(tidyverse)
library(ggraph)
library(tidygraph)
library(readxl)
library(corrplot)
library(ggrepel)
library(msigdbr)
library(fgsea)
library(ggvenn)
library(ComplexHeatmap)
library(ggVennDiagram)
library(venneuler)
library(sva)
library(DT)
library(tools)
library(tidyverse)
library(RSQLite)

library(edgeR)

txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

source('~/script/utils.r')
base_dir = '/research_jude/rgs01_jude/groups/jxugrp/home/common/Lab_Members/WenhuoHu/dataset/'

setwd(base_dir)


# design table

In [13]:
dsn = fread(text = 'GSE61566 GSM1508256 hs_CD34_1
GSE61566 GSM1508257 hs_CD34_2
GSE61566 GSM1508258 hs_CD34_3
GSE61566 GSM1508259 hs_BFU_1
GSE61566 GSM1508260 hs_BFU_2
GSE61566 GSM1508261 hs_BFU_3
GSE61566 GSM1508262 hs_CFU_1
GSE61566 GSM1508263 hs_CFU_2
GSE61566 GSM1508264 hs_CFU_3', header = F)
setnames(dsn, c('gse_id', 'gsm_id', 'sample_name'))

In [18]:
head(dsn)

gse_id,gsm_id,sample_name,srr_id
<chr>,<chr>,<chr>,<chr>
GSE61566,GSM1508256,hs_CD34_1,SRR1581233
GSE61566,GSM1508257,hs_CD34_2,SRR1581234
GSE61566,GSM1508258,hs_CD34_3,SRR1581235
GSE61566,GSM1508259,hs_BFU_1,SRR1581236
GSE61566,GSM1508260,hs_BFU_2,SRR1581237
GSE61566,GSM1508261,hs_BFU_3,SRR1581238


In [15]:
dsn[, srr_id := {system(paste0('esearch -db sra -query "', gsm_id, '" | efetch -format docsum | accn-at-a-time | grep -i srr | tr [:lower:] [:upper:] | tr "\n" "," '), intern = T)}, by = 1:nrow(dsn) ]


In [17]:
dsn[, srr_id := sub(',', '', srr_id) ]

In [ ]:
data_dir = '/home/whu78/WenhuoHu/dataset/gse616566/'
dsn[, sample_dir := paste0(data_dir, 'data/', gsm_id, '/') ]
inv = dsn[, {dir.create(sample_dir, recursive = T)}, by = 1:nrow(dsn) ]


In [23]:
setwd(data_dir)

In [27]:
source('~/script/utils.r')

In [42]:
dsn[, prefetch_cmd := paste0('singularity exec ', sif$sratool, ' prefetch ', srr_id, ' -O ', sample_dir, ' --max-size 40g')]
dsn[, fastqdump_cmd := paste0('singularity exec ', sif$sratool, ' fasterq-dump ', sample_dir, srr_id,  ' --outdir ', sample_dir)]


In [33]:
write(dsn$prefetch_cmd, file = 'run_fetch.sh') 
write(dsn$fastqdump_cmd, file = 'run_fetch.sh', append = T)


In [43]:
dsn[, fq_name := paste0(sample_dir, srr_id, '.fastq')]
file.exists(dsn[1, fq_name])


[1] TRUE

In [44]:
write.csv(dsn[, .(sample_name, fq_name)], file = 'samplesheet.csv', quote = F, row.names = F)